In [1]:
import pints
import pints.toy as toy
import pints.plot

import emupints
import emupints.plot as emuplt
import emupints.utils as emutils
import emupints.metrics as emumet

import numpy as np
import matplotlib.pyplot as plt
import GPy
from GPy import kern as kern

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import string
from itertools import product

## Specifying model

In [2]:
# Load problem from predefined ones
model = emupints.Problems.SIRModel
problem = emupints.Problems.load_problem(model)

problem['values'][:5]

array([[ 1.27153728,  0.3761075 ],
       [ 2.58756693, -0.09526055],
       [ 2.72011831,  2.65464822],
       [ 7.04147894,  3.79564764],
       [ 9.89153995,  4.78348756]])

In [3]:
# take all the required variables from the problem
n_parameters = problem['n_parameters']
log_likelihood = problem['log_likelihood']
log_prior = problem['log_prior']
log_posterior = problem['log_posterior']
real_parameters = problem['parameters']
pints_problem = problem['problem']
bounds = problem['bounds']
index_to_param_name = problem['param_names']

# Apply CMA-ES

In [4]:
# CMA-es
score = pints.SumOfSquaresError(pints_problem)
x0 = bounds.sample(1)[0]

found_parameters, found_value = pints.optimise(
    score,
    x0,
    None,
    bounds,
    method=pints.CMAES
)

Minimising error measure
using Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
Running in sequential mode.
Population size: 7
Iter. Eval. Best      Time m:s
0     7      2083.882   0:00.1
1     14     2083.882   0:00.1
2     21     1919.856   0:00.1
3     28     1463.567   0:00.1
20    147    566.5504   0:00.2
40    287    458.4774   0:00.3
60    427    456.2095   0:00.5
80    567    455.6646   0:00.6
100   707    455.6631   0:00.7
120   847    455.6631   0:00.7
140   987    455.6631   0:00.8
160   1127   455.6631   0:00.9
180   1267   455.6631   0:01.1
200   1407   455.6631   0:01.2
220   1547   455.6631   0:01.3
240   1687   455.6631   0:01.4
260   1827   455.6631   0:01.5
280   1967   455.6631   0:01.6
300   2107   455.6631   0:01.7
320   2240   455.6631   0:01.8
Halting: No significant change for 200 iterations.


# Use found parameters to sample trianing data

In [5]:
# training size
# use bigger training size for NNs because speed stays the same
train_size = 2000
test_size = 1000
sampling_std = 0.05 * found_parameters # take 5% around found parameters std

def normal_sampling(size, mean, std):
    return mean + np.random.randn(*size) * std

# create data for training and testing
# genera example using normal distribution around parametrs
train_X = normal_sampling((train_size, n_parameters), found_parameters, sampling_std)
train_y = np.apply_along_axis(log_likelihood, 1, train_X)

# if provide validation data to keras model also need to normalize it 
test_X = normal_sampling((test_size, n_parameters), found_parameters, sampling_std)
test_y = np.apply_along_axis(log_likelihood, 1, test_X)

## Creating GP and selecting kernel

In [6]:
emu = emupints.GPEmulator(log_likelihood,
                          train_X,
                          train_y,
                          input_scaler=StandardScaler(),
                         )

In [7]:
n_parameters = emu.n_parameters()


kernels = [
    kern.Linear(n_parameters),
    kern.RBF(n_parameters),
    kern.RatQuad(n_parameters),
    kern.MLP(n_parameters),
    kern.Matern52(n_parameters),
    kern.RatQuad(n_parameters) + kern.RBF(n_parameters),
    kern.Matern52(n_parameters) + kern.RBF(n_parameters),
    kern.RatQuad(n_parameters) + kern.RBF(n_parameters),
    kern.MLP(n_parameters) + kern.Matern52(n_parameters) + kern.RatQuad(n_parameters),
]

kernel_names = [
    'Linear',
    'RBF',
    'RatQuad',
    'MLP',
    'Matern',
    'RatQuad + RBF*RBF',
    'Matern52 + RBF * RBF',
    'RatQuad + RBF * RBF',
    'MLP + Matern + RatQuad'
]

# kernels that have been optimized for input data
trained_kernels = []

# possible optimizers: 
# ‘scg’, ‘lbfgs’, ‘tnc’
# can specify max number of iterations using max_iters
optimizer = "lbfgs"
max_iters = 500
emu.set_parameters(optimizer = optimizer)

In [8]:
# when the output data is normalized values of variance should be small
# hence ignore any kernel that has a subkernel 
# (i.e kernel that is a part of sum/product) 
# with variance > 1000
# set avoid_overfitting to False to stop this effect
avoid_overfitting = False
variance_threshold = 1000

# store prediction time for comparison
likelihood_time = %timeit -o log_likelihood(real_parameters)
likelihood_time = likelihood_time.average

577 µs ± 6.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
marginal_likelihoods = []
prediction_times = []

for kernel, kernel_name in zip(kernels, kernel_names):
    emu.set_parameters(kernel = kernel)
    emu.fit(optimize = False, normalizer = True)
    emu.optimize(max_iters = max_iters, messages = False)

    trained_kernel = emu.get_trained_kern()
    trained_kernels.append(trained_kernel)

    # time prediction
    emu_time =  %timeit -o emu(real_parameters)
    prediction_times.append(emu_time.average)

    ml = emu.get_log_marginal_likelihood()
    marginal_likelihoods.append(ml)
    print("{}: {:.2f}".format(kernel_name, ml))

2.45 ms ± 360 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Linear: -2009.41
2.31 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
RBF: 772.44
2.43 ms ± 63.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
RatQuad: 772.45
2.3 ms ± 76.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
MLP: 767.50
2.59 ms ± 63.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Matern: 768.67
3.01 ms ± 66.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
RatQuad + RBF*RBF: 773.17
3.31 ms ± 372 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Matern52 + RBF * RBF: 773.13
2.89 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
RatQuad + RBF * RBF: 773.17
KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [ ]:
best_kernel = None
best_score = -1 << 31
best_time = -1 << 31

# find kernel with highest log marginal likelihood
for kernel, score, emu_time in zip(trained_kernels, marginal_likelihoods, prediction_times):
    # ignore any overfitting kernel
    if avoid_overfitting and emutils.has_high_variance(kernel, threshold = variance_threshold):
        continue
    
    # avoid kernels that don't provide 5x speed up
    #if 5 * emu_time > likelihood_time:
    #    continue

    if score > best_score:
        best_kernel = kernel
        best_time = emu_time
        best_score = best_score
        
print('Prediction time: ', best_time)
best_kernel

In [ ]:
emu.set_parameters(kernel = best_kernel)
emu.fit(optimize = False, normalizer = True)
emu.optimize(max_iters = max_iters, messages = True)    

In [ ]:
emu.get_gp()

# Likelihood metrics

In [ ]:
emu_time = %timeit -r 50 -o emu(real_parameters)
print("Emulator prediction time (micro-seconds): {:5f}".format(emu_time.average * 1000000))

In [ ]:
likelihood_time = %timeit -r 50 -o log_likelihood(real_parameters)
print("Likelihood prediction time (micro-seconds): {:5f}".format(likelihood_time.average))

In [ ]:
pred_y = np.apply_along_axis(emu, 1, test_X)
print("Mean absolute error (mae): {:.5f}".format(emumet.mae(test_y, pred_y)))

In [ ]:
print("Mean percentage error (mape): {:.5f}".format(emumet.mape(test_y, pred_y) * 100))

# Comparing MCMC run times

In [ ]:
mcmc_method = pints.AdaptiveCovarianceMCMC

# MCMC parameters
num_chains = 3
max_iters = 10000

In [ ]:
# Create a posterior log-likelihood (log(likelihood * prior))
emu_log_posterior = pints.LogPosterior(emu, log_prior)

In [ ]:
xs = [
    found_parameters,
    found_parameters * 1.01,
    found_parameters * 0.99
]
xs

In [ ]:
emu_mcmc = pints.MCMCSampling(emu_log_posterior,
                              num_chains,
                              xs,
                              method = mcmc_method,
                             )
emu_mcmc.set_log_to_screen(False)
print('Running...')
emu_chains = emu_mcmc.run()
print('Done!')

In [ ]:
real_mcmc = pints.MCMCSampling(log_posterior, 
                               num_chains, 
                               xs, 
                               method = mcmc_method,
                               )
real_mcmc.set_max_iterations(max_iters)
real_mcmc.set_log_to_screen(False)
# Run!
print('Running...')
real_chains = real_mcmc.run()
print('Done!')

In [ ]:
pints.plot.trace(emu_chains)
plt.show()

In [ ]:
pints.plot.trace(real_chains)
plt.show()

# MCMC metrics

In [ ]:
print('Emulator R-hat:')
print(pints.rhat_all_params(emu_chains))

In [ ]:
print('Real R-hat:')
print(pints.rhat_all_params(real_chains))

In [ ]:
# percentage
chain = real_chains[0]

chain_mape = emumet.chain_mape(chain, emu_log, log_likelihood)
print("Chain mean absolute percentage error: {:.5f}".format(chain_mape))

In [ ]:
emu_mcmc_time = %timeit -o emu_mcmc.run()
print("Emulator MCMC time: {:.5f}".format(emu_mcmc_time.average))

In [ ]:
real_mcmc_time = %timeit -o real_mcmc.run()
print("MCMC time: {:.5f}".format(real_mcmc_time.average))

In [ ]:
emu_parameters, emu_parameters_std = emumet.estimate_parameters(emu_chains)
print("Emulator estimated parameters: ", emu_parameters)
print("Emulator parametet std: ", emu_parameters_std)

In [ ]:
real_parameters, real_parameters_std = emumet.estimate_parameters(real_chains)
print("Real estimated parameters: ", real_parameters)
print("Real parameters std: ", real_parameters_std)

# Plotting emulator surfaces around parameters

In [ ]:
# use smaller bounds to get an idea of the surface around real parameters
min_bounds = np.min(chain, axis = 0)
max_bounds = np.max(chain, axis = 0)
bounds = pints.RectangularBoundaries(min_bounds, max_bounds)

In [ ]:
# plot surfaces
if n_parameters >=3:
    fixed_parameters = emutils.fix_parameters(bounds)
    
    fig, ax = emuplt.plot_fixed_param_grid(
        emu,
        fixed_parameters,
        bounds,
        shape = (n_parameters, n_parameters - 1),
        contour = False,
        index_to_param_name=index_to_param_name,
        additional_log_likelihoods = [log_likelihood]
    )

    plt.show(fig)
    
# for row i we have parameter i fixed and some other parameter j, depending on the column, j != i

In [ ]:
print(bounds.lower())
print(bounds.upper())

In [ ]:
fig, ax = emuplt.plot_surface_fixed_param(
    emu,
    bounds,
#    fixed,
    index_to_param_name=index_to_param_name,
    contour=False,
    precision=2,
    additional_log_likelihoods=[log_likelihood]
)

In [ ]:
ax.view_init(45, 30 + 90 * 3)
fig.set_size_inches(10.5, 6)
fig